In [ ]:
# 교보문고 리뷰 데이터 크롤링 코드
# 카테고리별 1위부터 50위까지의 베스트셀러의 제목, 저자명, 리뷰 데이터 수집

In [3]:
# 필요 라이브러리 호출
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
import pandas as pd
import json

In [4]:
# 데이터 프레임 생성
# 열 : 카테고리(장르), 제목, 저자명, 리뷰
book_review_data = pd.DataFrame(columns=('category', 'title', 'author', 'review'))
book_review_data

,category,title,author,review


In [ ]:
# 3명이 카테고리를 열 별로 나누어 데이터 추출
# 1 : category_xpath = r'//*[@id="tabAnbCategorySub01"]/div[1]/div[1]/div[2]/div/div/div/div/div[1]/div/div[1]/ul/li[{}]/div[1]/a'.format(categorynumber)
# 2 : category_xpath = r'//*[@id="tabAnbCategorySub01"]/div[1]/div[1]/div[2]/div/div/div/div/div[1]/div/div[2]/ul/li[{}]/div[1]/a'.format(categorynumber)
# 3 : category_xpath = r'//*[@id="tabAnbCategorySub01"]/div[1]/div[1]/div[2]/div/div/div/div/div[1]/div/div[3]/ul/li[{}]/div[1]/a'.format(categorynumber)

In [ ]:
#동적 데이터 크롤링 코드
for categorynumber in range(1, 12):
    #1. 사이트 접속
    driver = webdriver.Chrome()
    driver.get('https://www.kyobobook.co.kr/') #교보문고 메인 화면 접근
    act = ActionChains(driver)
    # 카테고리 버튼을 클릭하여 오픈
    driver.find_element(By.XPATH, r'/html/body/div[3]/header/div[3]/nav/div[1]/button').click()
    #각 카테고리의 이름을 모른다는 전제 하에, li다음의 숫자가 변경됨을 이용해 for문 loop로 xpath를 변경
    #이때, 카테고리명을 '소설', '에세이' 등으로 직접 지정해 path를 찾을 수도 있음
    category_xpath = r'//*[@id="tabAnbCategorySub01"]/div[1]/div[1]/div[2]/div/div/div/div/div[1]/div/div[1]/ul/li[{}]/div[1]/a'.format(categorynumber)
    driver.find_element(By.XPATH, category_xpath).click() #카테고리 접속
    # 여러 개의 tab_text버튼 중 <베스트셀러>를 클릭
    for element in driver.find_elements(By.XPATH,'//span[@class="tab_text"]'):
        if element.text == '베스트셀러':
            element.click()
    # (1)카테고리명 추출
    category = driver.find_element(By.CLASS_NAME, 'snb_title').text
    
    
    #2. 베스트셀러 50위까지의 상세페이지 url 추출
    book_page_urls = [] # 책의 상세 웹페이지 주소를 추출하여 리스트에 저장
    # 베스트셀러가 한 페이지에 20개씩 노출되는 것이 default
    # 50개 보기로 변경 시, 베스트셀러 순서가 아닌 오늘의 선택 코너 책이 추출됨
    for _ in range(3):
        time.sleep(2)
        bsObject = BeautifulSoup(driver.page_source, 'html.parser')
        # find_all 기준을 달리 하면 엉뚱하게 추출되어 prod_viewer_control로 추출
        for item in bsObject.find_all('div', 'prod_viewer_control'):
            url = item.select('a')[0].get('href')
            book_page_urls.append(url)
        time.sleep(2)
        #다음 페이지로 넘어가는 버튼 클릭
        while True:
            try:
                time.sleep(2)
                driver.find_element(By.CLASS_NAME, 'btn_page.next').click()
                break
            except:
                # 다음 페이지로 넘어가는 버튼을 클릭할때까지 밑으로 스크롤
                # document.body.scrollHeight으로 맨 밑까지 한 번에 내려가지 않을 때가 있어 try except문 사용
                driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    book_page_urls = book_page_urls[:50] # 3번 페이지를 넘겨 60개의 책을 가져온 뒤, 50위까지 사용
    
    
    #3. 책 각각의 상세 페이지에 접속해 필요한 정보를 추출
    for index, book_page_url in enumerate(book_page_urls):
        df_index = (categorynumber-1)*50 + index
        finish = False #리뷰의 마지막 페이지인지를 확인하는 boolean 변수
        driver.get(book_page_url)
        time.sleep(2)
        # (2)책 제목
        title = driver.find_element(By.CLASS_NAME, 'prod_title').text
        # (3)저자 이름
        author = driver.find_element(By.CLASS_NAME, 'author>a').text
        # (4)리뷰 (리스트 형태로 저장)
        review = []
        driver.execute_script("window.scrollTo(0, 200)") #review 버튼 보이는 위치까지 스크롤 다운
        driver.find_element(By.CLASS_NAME, 'btn_go_review').click() #리뷰 창으로 스크롤을 한 번에 내리는 버튼 클릭
        time.sleep(2)
        review_location = driver.execute_script("return window.pageYOffset") #리뷰 창의 위치 저장
        #리뷰의 마지막 페이지일때까지 리뷰 추출을 반복
        while (finish == False):
            scroll_location = review_location + 1000
            for i in range(1, 11):
                # 한 페이지에 있는 댓글 10개 가져오기
                # try문으로 처리한 이유 : 마지막 페이지엔 댓글이 10개 미만일 수 있음.
                try:
                    xpath = '//*[@id="ReviewList1"]/div[3]/div[2]/div/div[1]/div[{}]/div[2]/div/div/div/div'.format(i)
                    review.append(driver.find_element(By.XPATH, xpath).text)
                except:pass
            # 스크롤을 아래로 내려서 리뷰 다음 페이지로 넘어가도록 함.
            # 문제 : 댓글 리뷰창 바로 아래에 리뷰창을 넘기는 버튼, 그 밑에 '문장수집'을 옆으로 넘기는 버튼이 존재
            # 스크롤을 한 번에 내리면, 리뷰창이 아닌 문장수집이 넘어갈 가능성이 높음
            # 따라서 스크롤을 300씩 천천히 내리도록 코드를 작성
            while True:
                driver.execute_script("window.scrollTo(0, {})".format(scroll_location))
                time.sleep(2)
                try:
                    next_button = driver.find_element(By.CLASS_NAME, 'btn_page.next')
                    # next_button이 동작할 수 있으면 클릭하여 리뷰 페이지를 넘김
                    if next_button.is_displayed() and next_button.is_enabled():
                        next_button.click()
                        time.sleep(2)
                        break
                    # next_button이 동작하지 않게 돼있으면,
                    # 즉 disabled로 돼있으면 마지막 페이지라 더이상 못넘긴다는 의미
                    # 따라서 while loop 끝내기 위해 finish = True로 변경
                    if next_button.is_enabled() == False:
                        finish = True
                        break
                # next_button을 찾을 때까지 스크롤을 내림
                except :
                    time.sleep(2)
                    scroll_location += 300
                    pass

        book_review_data.loc[df_index] = [category, title, author, review]
        print(book_review_data)

In [ ]:
# 오류가 발생했다면 끊긴 지점에서 다시 작동
# ex : 22번째 책에서 오류가 발생
for index, book_page_url in enumerate(book_page_urls[22:]):
    df_index = (categorynumber-1)*50 + index + 22
    finish = False
    driver.get(book_page_url)
    time.sleep(2)
    title = driver.find_element(By.CLASS_NAME, 'prod_title').text
    author = driver.find_element(By.CLASS_NAME, 'author>a').text
    review = []
    driver.execute_script("window.scrollTo(0, 200)")
    driver.find_element(By.CLASS_NAME, 'btn_go_review').click()
    time.sleep(2)
    review_location = driver.execute_script("return window.pageYOffset")
    while (finish == False):
        scroll_location = review_location+1000
        for i in range(1, 11):
            try:
                xpath = '//*[@id="ReviewList1"]/div[3]/div[2]/div/div[1]/div[{}]/div[2]/div/div/div/div'.format(i)
                review.append(driver.find_element(By.XPATH, xpath).text)
            except:pass
        while True:
            driver.execute_script("window.scrollTo(0, {})".format(scroll_location))
            time.sleep(2)
            try:
                next_button = driver.find_element(By.CLASS_NAME, 'btn_page.next')
                if next_button.is_displayed() and next_button.is_enabled():
                    next_button.click()
                    time.sleep(2)
                    break
                if next_button.is_enabled() == False:
                    finish = True
                    break
            except :
                time.sleep(2)
                scroll_location += 300
                pass

    book_review_data.loc[df_index] = [category, title, author, review]
    print(book_review_data)

In [47]:
book_review_data.to_json('book_review_data.json')

In [9]:
#저장된 데이터 확인
book_review_data = pd.read_json('book_review_data.json')
book_review_data

,category,title,author,review
0,소설,돌연한 출발,프란츠 카프카,"[얼마나 재밌을지 기대되요, 표지도 너무 이쁘고 책의날 카프카를 읽을수 있었네요, ..."
1,소설,메리골드 마음 세탁소,윤정은,"[마음이 힐링 되는 느낌이에요, 매우만족합니다\n추천드려요, 저에게도 필요해요\n마..."
2,소설,스즈메의 문단속,신카이 마코토,"[영화랑 또 다른 매력이 느껴져서 좋아요!!, 아이들이 재미있게 보네요, 너무 재미..."
3,소설,2023 제14회 젊은작가상 수상작품집,이미상,"[재미있게 읽었습니다, 문학동네가 배가 부른걸까?\n그래도 대상받은것이라 인내심을 ..."
4,소설,불편한 편의점(40만부 기념 벚꽃 에디션),김호연,"[읽다보니 시간 잘가고 재밌어요, 순식간에 읽어 내려감, 사람과 사람간의 얘기가 좋..."
...,...,...,...,...
545,역사/문화,썬킴의 세계사 완전 정복,썬킴,"[아이가 재미있게 잘 읽고 있어요., 재미있게 잘 읽었습니다, 재미있게 잘 읽었습니..."
546,역사/문화,시민의 한국사 2: 근현대편,한국역사연구회,"[흐름에 따라 읽기 편했어요, 두꺼운 책이지만 추천합니다. 우리의 근현대 이야기!,..."
547,역사/문화,거의 모든 전쟁의 역사,제러미 블랙,"[잘받았습니다.감사합니다., 내용이 너무 방대하고 설명이 친절하진 않아요, 잘받았습..."
548,역사/문화,일본의 굴레,태가트 머피R,[일본에 대한 단순한 관심으로 구매했는데 이책을 읽고 40년 넘게 살면서 궁금했던 ...


In [30]:
# 리뷰 데이터 확인
# 소설 베스트셀러 1위 돌연한 출발 - 프란츠 카프카
book_review_data['review'][0]

['얼마나 재밌을지 기대되요',
 '표지도 너무 이쁘고 책의날 카프카를 읽을수 있었네요',
 '최고의책이에요 조아요',
 '프라하에 가고싶다. 죽기 전에',
 '돌연한 출발 천천히 곱씹으면서 읽고 있어요',
 '카프카 좋습니다 읽어보세요',
 '카프카 좋습니다.',
 '난해하지만 믿고 보는 카프카',
 '그림이 너무 예뻐서 더욱 좋아요',
 '좋아하는 문장이 있는 책이라 구입했습니다',
 '너무 좋고 재미있어요',
 '카프카의 편지부터 단편까지 아주 알차게 이루어져 있습니다. 읽으면 읽을수록 카프카를 깊게 알게되는 것 같기고 하고 어쩌면 그 반대일 수도 있다는 느낌이 드는 이야기의 향연..',
 '그림이랑 구성이 넘 잘 되어 있네요',
 '그림이랑 구성이 너무 잘 되어 있네요',
 '네 좋아요 힐링이에요',
 '좋아요! 아주 좋아요!',
 '좋아요! 아주 좋아요!',
 '',
 '',
 'ㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓㅓ',
 '리커버는 구매해야죠. 책이 너무 예쁘네요',
 '카프카의 글과 그림들 더이상 무슨 설명이 필요할까',
 '감사감사감사감사감사',
 '고전을 다시 보는 재미 좋아요',
 '잘 보겠습니다. 감사합니다.',
 '프란츠 카프카 작가 이름자체가 보증이 됩니다 예쁜 표지만큼이나기대되네요.',
 '좋아요',
 '재밌네요 깊이가 있어요',
 '',
 '감동적인 스토리 한숨에 읽었어요',
 '아직보진안았지만 기대됩니다..',
 '단편집이 있지만 소장용으로 구입. 예뻐요',
 '기대됩니다. 잘읽을게요',
 '기대되요 어서보고싶네요',
 '',
 '너무너무기대가됩니다',
 '기대되요 고마워요 감사합니다',
 '출발준비완료했습니다.',
 '소송을 읽은 이후로 카프카에 관심이 생겼는데 마침 이런 책이 출간되어서 바로 샀습니다. 감사합니다.',
 '프란츠카프카의 책은 첨인데 기대되네요',
 '카프카의 필력이 고스란히 전달되어 오는 소장할만한 서적이네요.',
 '간편하게 읽을 수 있어서 좋습니다',
 '카프카와의 만남 기대하며....',
 '돌연한 출발 금방 읽혀요',


In [42]:
get_review_count = lambda x: len(x)
book_review_data['review_count'] = book_review_data['review'].apply(get_review_count) #리뷰 개수 열
book_review_data

,category,title,author,review,review_count
0,소설,돌연한 출발,프란츠 카프카,"[얼마나 재밌을지 기대되요, 표지도 너무 이쁘고 책의날 카프카를 읽을수 있었네요, ...",110
1,소설,메리골드 마음 세탁소,윤정은,"[마음이 힐링 되는 느낌이에요, 매우만족합니다\n추천드려요, 저에게도 필요해요\n마...",191
2,소설,스즈메의 문단속,신카이 마코토,"[영화랑 또 다른 매력이 느껴져서 좋아요!!, 아이들이 재미있게 보네요, 너무 재미...",527
3,소설,2023 제14회 젊은작가상 수상작품집,이미상,"[재미있게 읽었습니다, 문학동네가 배가 부른걸까?\n그래도 대상받은것이라 인내심을 ...",91
4,소설,불편한 편의점(40만부 기념 벚꽃 에디션),김호연,"[읽다보니 시간 잘가고 재밌어요, 순식간에 읽어 내려감, 사람과 사람간의 얘기가 좋...",2905
...,...,...,...,...,...
545,역사/문화,썬킴의 세계사 완전 정복,썬킴,"[아이가 재미있게 잘 읽고 있어요., 재미있게 잘 읽었습니다, 재미있게 잘 읽었습니...",56
546,역사/문화,시민의 한국사 2: 근현대편,한국역사연구회,"[흐름에 따라 읽기 편했어요, 두꺼운 책이지만 추천합니다. 우리의 근현대 이야기!,...",41
547,역사/문화,거의 모든 전쟁의 역사,제러미 블랙,"[잘받았습니다.감사합니다., 내용이 너무 방대하고 설명이 친절하진 않아요, 잘받았습...",53
548,역사/문화,일본의 굴레,태가트 머피R,[일본에 대한 단순한 관심으로 구매했는데 이책을 읽고 40년 넘게 살면서 궁금했던 ...,77


In [43]:
book_review_data

,category,title,author,review,review_count
0,소설,돌연한 출발,프란츠 카프카,"[얼마나 재밌을지 기대되요, 표지도 너무 이쁘고 책의날 카프카를 읽을수 있었네요, ...",110
1,소설,메리골드 마음 세탁소,윤정은,"[마음이 힐링 되는 느낌이에요, 매우만족합니다\n추천드려요, 저에게도 필요해요\n마...",191
2,소설,스즈메의 문단속,신카이 마코토,"[영화랑 또 다른 매력이 느껴져서 좋아요!!, 아이들이 재미있게 보네요, 너무 재미...",527
3,소설,2023 제14회 젊은작가상 수상작품집,이미상,"[재미있게 읽었습니다, 문학동네가 배가 부른걸까?\n그래도 대상받은것이라 인내심을 ...",91
4,소설,불편한 편의점(40만부 기념 벚꽃 에디션),김호연,"[읽다보니 시간 잘가고 재밌어요, 순식간에 읽어 내려감, 사람과 사람간의 얘기가 좋...",2905
...,...,...,...,...,...
545,역사/문화,썬킴의 세계사 완전 정복,썬킴,"[아이가 재미있게 잘 읽고 있어요., 재미있게 잘 읽었습니다, 재미있게 잘 읽었습니...",56
546,역사/문화,시민의 한국사 2: 근현대편,한국역사연구회,"[흐름에 따라 읽기 편했어요, 두꺼운 책이지만 추천합니다. 우리의 근현대 이야기!,...",41
547,역사/문화,거의 모든 전쟁의 역사,제러미 블랙,"[잘받았습니다.감사합니다., 내용이 너무 방대하고 설명이 친절하진 않아요, 잘받았습...",53
548,역사/문화,일본의 굴레,태가트 머피R,[일본에 대한 단순한 관심으로 구매했는데 이책을 읽고 40년 넘게 살면서 궁금했던 ...,77
